In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 우리가 데이터를 가져올 웹 페이지입니다.
faker_opgg_url = 'https://www.op.gg/summoner/userName=Hide%20on%20bush'

# 페이지의 html 코드를 faker_html로 저장합니다.
faker_html = requests.get(faker_opgg_url).text

# html 형식에 맞춰 파싱(parsing; 추후 이용하기 쉽도록 쪼개기)합니다.
faker_soup = BeautifulSoup(faker_html, "html.parser")

In [2]:
faker_soup

 <!DOCTYPE html>

<!--
	  ,ad8888ba,   88888888ba         ,ad8888ba,    ,ad8888ba,
	 d8"'    `"8b  88      "8b       d8"'    `"8b  d8"'    `"8b
	d8'        `8b 88      ,8P      d8'           d8'
	88          88 88aaaaaa8P'      88            88
	88          88 88""""""'        88      88888 88      88888
	Y8,        ,8P 88               Y8,        88 Y8,        88
	 Y8a.    .a8P  88                Y8a.    .a88  Y8a.    .a88
	  `"Y8888Y"'   88     dev@op.gg   `"Y88888P"    `"Y88888P"

		 Copyright © 2013-2021 OP.GG. All rights reserved.
-->
<html lang="en_US">
<head>
<script>
		(function(t, l) {
			try {
				let w = window,
						d = document,
						s = d.createElement("script"),
						f = d.getElementsByTagName("script")[0];
				w[t] =
						w[t] ||
						function() {
							(w["_rgea"] = w["_rgea"] || [["uts", new Date()]]).push(
									Array.prototype.slice.call(arguments)
							);
							w.RGEA && w.RGEA.p();
						};
				s.type = "text/javascript";
				s.async = true;
				s.defe

In [3]:
# 결과가 들어갈 빈 리스트를 만듭니다.
faker_recent_champions = []
faker_recent_kills = []
faker_recent_deaths = []
faker_recent_assists = []
faker_recent_results = []

In [4]:
# 전체 html 코드 중 우리가 원하는 selector를 만족하는 것만 가져오기
faker_recent_games_html = faker_soup.select('div.GameItemList div.GameItemWrap')

# 그렇게 가져온 html 코드의 element 개수 == 한 페이지에 최초로 로딩된 최근 게임 수 출력
recent_game_len = len(faker_recent_games_html)
print(recent_game_len)

20


In [5]:
type(faker_recent_games_html)

list

- `select()` 사용하면 리스트 형태로 저장

In [6]:
# 각 게임에 대해 웹 페이지에 기재된 스탯을 찾아서(selector 사용) 결과 리스트에 append하기
for i in range(recent_game_len):
    faker_recent_champions.append(''.join(list(faker_recent_games_html[i].select('div.ChampionName')[0].
                                               stripped_strings)))
    faker_recent_kills.append(list(faker_recent_games_html[i].select('div.KDA div.KDA span.Kill')[0].
                                               stripped_strings)[0])
    faker_recent_deaths.append(list(faker_recent_games_html[i].select_one('div.KDA div.KDA span.Death').
                                               stripped_strings)[0])
    faker_recent_assists.append(list(faker_recent_games_html[i].select('div.KDA div.KDA span.Assist')[0].
                                               stripped_strings)[0])
    faker_recent_results.append(list(faker_recent_games_html[i].select('div.GameStats div.GameResult')[0].
                                               stripped_strings)[0])
#     아래 코드는 data-game-result라는 attr 가져오는 방법 다만 다시하기도 lose라 되있어서 사용 x
#     faker_recent_results.append(faker_recent_games_html[i].select_one('div.GameItem')['data-game-result'])

In [7]:
# DataFrame으로 변환 후 출력
faker_recent_df = pd.DataFrame([faker_recent_champions,
                                faker_recent_results,
                                faker_recent_kills,
                                faker_recent_deaths,
                                faker_recent_assists],
                               index = ['champion', 'result', 'kills', 'deaths', 'assists']).T
faker_recent_df

,champion,result,kills,deaths,assists
0,Orianna,Victory,10,3,18
1,Lee Sin,Victory,8,4,14
2,Anivia,Defeat,7,5,10
3,LeBlanc,Victory,4,2,4
4,Blitzcrank,Victory,4,1,17
5,Orianna,Defeat,3,3,0
6,Orianna,Defeat,2,7,3
7,Tristana,Victory,9,8,9
8,Vayne,Victory,8,5,10
9,Aphelios,Victory,5,7,2


In [8]:
faker_recent_games_html[0].select('div.ChampionName')[0].get_text()

'\nOrianna\n'

In [9]:
list(faker_recent_games_html[0].select('div.ChampionName')[0].strings)

['\n', 'Orianna', '\n']

In [10]:
list(faker_recent_games_html[0].select('div.ChampionName')[0].stripped_strings)

['Orianna']

In [11]:
faker_recent_games_html[0].find("div", attrs={"class":"ChampionName"})

<div class="ChampionName">
<a href="/champion/orianna/statistics" target="_blank">Orianna</a>
</div>